In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9042c858779c709c2c33d66f562bf787751c97fee2bad5158375132eb90c216e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("pyspark").getOrCreate()
print(spark)

In [4]:
df=spark.read.csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv",header=True,inferSchema=True)

In [5]:
df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [7]:
df=df.drop("customerID")

In [8]:
df.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|

In [40]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [49]:
inputCols=["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","Churn"]
outputcols=["gender1","Partner1","Dependents1","PhoneService1","MultipleLines1","InternetService1","OnlineSecurity1","OnlineBackup1","DeviceProtection1","TechSupport1","StreamingTV1","StreamingMovies1","Contract1","PaperlessBilling1","PaymentMethod1","Churn1"]
si=StringIndexer(inputCols=inputCols,outputCols=outputcols)

In [50]:
va=VectorAssembler(inputCols=outputcols,outputCol="features")

In [51]:
df.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|

In [57]:
from pyspark.ml.classification import LogisticRegression
lr=LogisticRegression(featuresCol="features",labelCol="Churn1")

In [58]:
from pyspark.ml import Pipeline

In [67]:
pipeline=Pipeline(stages=[si,va,lr])

In [68]:
train_df,test_df=df.randomSplit([0.7,0.3])

In [71]:
model = pipeline.fit(train_df)
predictions=model.transform(test_df)

In [72]:
predictions.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-------+--------+-----------+-------------+--------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+---------+-----------------+--------------+------+--------------------+--------------------+--------------------+----------+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|gender1|Partner1|Dependents1|PhoneService1|MultipleLines1|InternetService1|OnlineSecurity1|OnlineBackup1|DeviceProtection1|TechSupport1|StreamingTV1|StreamingMovies1|Contract1|P